In [1]:
import os

from typing import Any, List, Dict, Mapping, Tuple, Union, Optional
import copy
from dataclasses import dataclass, field
import json
import pathlib
from typing import Dict, Optional, Sequence
import pickle
import tqdm

import numpy as np
import torch
import torch.nn.functional as F
from torch.nn.utils.rnn import pad_sequence
from torch.utils.data import Dataset
import transformers
from transformers import Trainer
from transformers.trainer_pt_utils import LabelSmoother

from transformers import LlamaTokenizer, LlamaForCausalLM

In [2]:
class FinQAFinetuneDataset(Dataset):
    def __init__(self, data_path, tokenizer):
        self.tokenizer = tokenizer
        self.tokenizer.pad_token_id = self.tokenizer.unk_token_id

        with open(data_path, "rb") as f:
            self.data = pickle.load(f)

    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, idx):
        input_ids = self.data[idx]["input_ids"]
        labels = self.data[idx]["labels"]
        attention_mask = self.data[idx]["attention_mask"]
        return dict(
            input_ids=input_ids,
            labels=labels,
            attention_mask=attention_mask,
        )


In [3]:
tokenizer = LlamaTokenizer.from_pretrained("decapoda-research/llama-65b-hf")

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'LLaMATokenizer'. 
The class this function is called from is 'LlamaTokenizer'.


In [4]:
dataset = FinQAFinetuneDataset(data_path="./llama_finetune_dataset.pkl", tokenizer=tokenizer)

FileNotFoundError: [Errno 2] No such file or directory: './llama_finetune_dataset.pkl'

In [5]:
with open("../finetune_data_prepared.jsonl") as f:
    data = [json.loads(line) for line in f]

In [6]:
all_examples = []

for item in tqdm.tqdm(data):
    prompt = item["prompt"]
    answer = item["completion"]

    prompt_ids = tokenizer.encode(prompt, add_special_tokens=False)
    answer_ids = tokenizer.encode(answer, add_special_tokens=False) + [tokenizer.eos_token_id]

    input_ids = torch.tensor(prompt_ids + answer_ids, dtype=torch.long)

    if len(input_ids) > 2048:
        print("skipping", len(input_ids))
        continue

    labels = input_ids.clone()
    labels[: len(prompt_ids)] = -100

    all_examples.append(
        dict(
            input_ids=input_ids,
            labels=labels,
            attention_mask=torch.ones_like(input_ids, dtype=torch.bool),
        )
    )

  1%|▏         | 92/6203 [00:00<00:25, 236.09it/s]

skipping 2585
skipping 2409


  4%|▍         | 244/6203 [00:01<00:24, 245.44it/s]

skipping 2350


  6%|▌         | 345/6203 [00:01<00:24, 239.85it/s]

...

skipping 2100
skipping 2540
skipping 2328


 99%|█████████▉| 6149/6203 [00:26<00:00, 219.14it/s]

skipping 2265
skipping 2061
skipping 2336


100%|██████████| 6203/6203 [00:26<00:00, 235.04it/s]

skipping 2381
skipping 2287
skipping 2389


In [8]:
with open("./llama_finetune_dataset.pkl", "wb") as f:
    pickle.dump(all_examples, f)